In [1]:
from MCMC_newmodel_3params import run_chain
import multiprocessing
def run_parallel_chains(true_params, num_iterations, num_chains):
    pool = multiprocessing.Pool(processes=num_chains)
    args_list = [(true_params, num_iterations, i) for i in range(num_chains)]
    results = pool.map(run_chain, args_list)
    pool.close()
    pool.join()
    chain_accepted_parameters = [result[0] for result in results]
    counts = [result[1] for result in results]
    dists = [result[2] for result in results]
    return chain_accepted_parameters, counts, dists

In [48]:
ap,c,d=run_parallel_chains(true_params,50000,1)

10000th iterations done, previously acceptance rate = 0.5522, threshold is set to 589.2125345094432
10000th iterations done, previously acceptance rate = 1.9998, threshold is set to 649.4694862915196
10000th iterations done, previously acceptance rate = 0.9999, threshold is set to 664.9395310459961
15000th iterations done, previously acceptance rate = 0.4716, threshold is set to 471.1726611438824
10000th iterations done, previously acceptance rate = 0.9999, threshold is set to 661.3902483143186
20000th iterations done, previously acceptance rate = 0.4282, threshold is set to 406.0047281019718
20000th iterations done, previously acceptance rate = 1.1156, threshold is set to 600.5409204195926
20000th iterations done, previously acceptance rate = 0.6607, threshold is set to 617.5794224948676
25000th iterations done, previously acceptance rate = 0.4766, threshold is set to 319.26409700029876
20000th iterations done, previously acceptance rate = 0.6386, threshold is set to 616.5686889362164

KeyboardInterrupt: 

30000th iterations done, previously acceptance rate = 0.4764, threshold is set to 528.4513340802852
35000th iterations done, previously acceptance rate = 0.5022, threshold is set to 188.8142530466227
30000th iterations done, previously acceptance rate = 0.4487, threshold is set to 543.441296327315
40000th iterations done, previously acceptance rate = 0.4694, threshold is set to 156.2059089217339


In [13]:
import multiprocessing
import numpy as np

from MCMC_newmodel_3params import run_chain1
result_list = []
def append_result(result):
    # This is called whenever foo_pool(i) returns a result.
    # result_list is modified only by the main process, not the pool workers.
    result_list.append(result)

def run_parallel_chains(true_params, num_iterations, num_chains):
    chain_parameters = [[(true_params, num_iterations)] for _ in range(num_chains)]
    # Create a multiprocessing Pool
    with multiprocessing.Pool() as pool:
        # Use pool.map to run the chains in parallel
        results = pool.starmap(run_chain1, chain_parameters)
    pool.close()
    pool.join()
    params=np.array([result[0] for result in results])
    acceptance=np.array([result[1] for result in results])
    distance=np.array([result[2] for result in results])
    return(params, acceptance, distance)

In [15]:
pa, ac, di=run_parallel_chains(true_params=[200.,200.,200.], num_iterations=50000, num_chains=4)

10000th iterations done, previously acceptance rate = 0.0, threshold is set to 600
10000th iterations done, previously acceptance rate = 0.8941, threshold is set to 443.14635890395346
10000th iterations done, previously acceptance rate = 0.8898, threshold is set to 440.4235574630973
10000th iterations done, previously acceptance rate = 0.8876, threshold is set to 464.36760982516284
20000th iterations done, previously acceptance rate = 0.4069, threshold is set to 340.08020601973817
20000th iterations done, previously acceptance rate = 0.0, threshold is set to 600
20000th iterations done, previously acceptance rate = 0.4099, threshold is set to 334.9925165059373
20000th iterations done, previously acceptance rate = 0.4366, threshold is set to 373.93072871241026
30000th iterations done, previously acceptance rate = 0.4281, threshold is set to 225.48510206913696
30000th iterations done, previously acceptance rate = 0.0, threshold is set to 600
30000th iterations done, previously acceptance

In [11]:
results

NameError: name 'results' is not defined

# Additional Posterior Checks as SBIBM

The median distance should be considered a mere check rather than a metric and it
does not necessarily test the structure of the estimated posterior.

Median L2 distance from posterior samples and x_o

In [ ]:
import numpy as np
import torch


def median_distance(
    predictive_samples: torch.Tensor,
    observation: torch.Tensor,
) -> torch.Tensor:
    """Compute median distance

    Uses NumPy implementation, see [1] for discussion of differences.

    Args:
        predictive_samples: Predictive samples
        observation: Observation

    Returns:
        Median distance

    [1]: https://github.com/pytorch/pytorch/issues/1837
    """
    assert predictive_samples.ndim == 2
    assert observation.ndim == 2

    l2_distance = torch.norm((observation - predictive_samples), dim=-1)
    return torch.tensor([np.median(l2_distance.numpy()).astype(np.float32)])